# Projeto Final | Sistema para Edição de imagens e áudios

<div style="background-color: #f2f2f2; text-align: center; padding: 10px;">
  <h3>Script para edição de áudios</h3>
</div>

%------------------------------------------------------------------------------------------------------%<br>
<br>
``Autores:`` Andrea Elias, Anthony Heimlich, Éverton Donato, Julia Midori e Luana Kruger  <br>
<br>
``Instituição:`` ADA Tech<br>
<br>
``Projeto:`` Santander Coders 2023.2<br>
<br>
``Descrição:`` Este código implementa um sistema de edição de áudios e registra informações sobre as manipulações feitas em um arquivo de log<br>
<br>
``Repositório GitHub:`` https://github.com/luana-kruger/TP-I_Santander_Coders23  <br>
<br>

%------------------------------------------------------------------------------------------------------%

## Carregar bibliotecas

In [58]:
import csv
import os
from datetime import datetime

import numpy as np

import scipy.io.wavfile as wav
from IPython.display import Audio

## Carregar Funções

### Função para criar ou abrir arquivo de log


In [2]:
def criar_ou_abrir_arquivo_log():

    arquivo_existe = os.path.isfile('\logs\log_audio.csv')
    modo = 'a' if arquivo_existe else 'w'
    arquivo_log = open('\logs\log_audio.csv', modo, newline='')
    
    return csv.DictWriter(arquivo_log, 
                          fieldnames=['Data e Hora', 'Tipo de Processamento', 'Tempo de Processamento', 
                                      'Nome do Arquivo', 'Tamanho do Arquivo', 'Resultado do Processamento', 
                                      'Mensagem de Erro']
                        )


### Função para registrar log

In [3]:
def registrar_log(arquivo_log, tipo_processamento, tempo_processamento, nome_arquivo, tamanho_arquivo, resultado, mensagem_erro):
    
    data_hora = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    arquivo_log.writerow({'Data e Hora': data_hora,  
                          'Tipo de Processamento': tipo_processamento, 
                          'Tempo de Processamento': tempo_processamento, 
                          'Nome do Arquivo': nome_arquivo, 
                          'Tamanho do Arquivo': tamanho_arquivo, 
                          'Resultado do Processamento': resultado, 
                          'Mensagem de Erro': mensagem_erro})


### Função Ler arquivo WAV

In [30]:
def ler_arquivo_wav(path, arquivo_log):
    
    tipo_process = 'Leitura de Arquivo WAV'
    nome_arquivo = os.path.basename(path)
    tamanho_arquivo = os.path.getsize(path) / 1024  # Tamanho em KB

    try:
        tempo_inicio = datetime.now()

        rate, audio_data = wav.read(path)

        tempo_fim = datetime.now()

        # Tempo em milissegundos
        tempo_processamento = (tempo_fim - tempo_inicio).total_seconds() * 1000

        registrar_log(arquivo_log, tipo_process, tempo_processamento, nome_arquivo, tamanho_arquivo, 'Sucesso', '')
        
        return rate, audio_data
    
    except Exception as erro:

        registrar_log(arquivo_log, tipo_process, None, nome_arquivo, tamanho_arquivo, 'Erro', str(erro))
        
        return None, None

### Função Convolução do áudio


In [57]:
## Convolução é uma operação matemática comum em processamento de sinais que combina duas 
## funções para produzir uma terceira, representando como uma é afetada pela outra ao longo do tempo.

def convolucao(audio_data, path, arquivo_log):

    tipo_process = 'Convolução de Áudio'
    nome_arquivo = os.path.basename(path)
    tamanho_arquivo = os.path.getsize(path) / 1024  # Tamanho em KB

    try:

        tempo_inicio = datetime.now()

        f_trans = np.poly1d([1], True)
        audio_final = np.convolve(audio_data, f_trans)

        tempo_fim = datetime.now()
        
        # Tempo em milissegundos
        tempo_processamento = (tempo_fim - tempo_inicio).total_seconds() * 1000 

        registrar_log(arquivo_log, tipo_process, tempo_processamento, nome_arquivo, tamanho_arquivo, 'Sucesso', '')
        
        return audio_final
    
    except Exception as erro:

        registrar_log(arquivo_log, tipo_process, None, nome_arquivo, tamanho_arquivo, 'Erro', str(erro))

        return None


### Função Normalização de áudio

In [6]:
## Normalizar o áudio para ajustar os níveis de amplitude

def normalizar_audio(audio_data, path, arquivo_log):

    tipo_process = 'Normalização de Áudio'
    nome_arquivo = os.path.basename(path)
    tamanho_arquivo = os.path.getsize(path) / 1024  # Tamanho em KB

    try:
        tempo_inicio = datetime.now()

        audio_normalizado = audio_data / np.max(np.abs(audio_data))

        tempo_fim = datetime.now()
        
        # Tempo em milissegundos
        tempo_processamento = (tempo_fim - tempo_inicio).total_seconds() * 1000 

        registrar_log(arquivo_log, tipo_process, tempo_processamento, nome_arquivo, tamanho_arquivo, 'Sucesso', '')
        
        return audio_normalizado
    
    except Exception as erro:

        registrar_log(arquivo_log, tipo_process, None, nome_arquivo, tamanho_arquivo, 'Erro', str(erro))

        return None

### Função Corte de áudio

In [4]:
# Recortar parte do áudio, selecionando apenas uma parte específica.

### Função Concatenação de áudio

In [8]:
# Juntar dois ou mais arquivos de áudio em um único arquivo.


### Função Filtragem 

In [9]:
# Aplicar filtros, como passa-baixa ou passa-alta, para alterar as características do som.

## Teste

In [43]:
## Parametros

path = './audio_test.wav'

In [8]:
Audio(path)

In [56]:
with open('..\logs\log_audio.csv', 'w', newline='') as arquivo_log:
    writer = csv.DictWriter(arquivo_log, fieldnames=['Data e Hora', 'Tipo de Processamento', 'Tempo de Processamento', 'Nome do Arquivo', 'Tamanho do Arquivo', 'Resultado do Processamento', 'Mensagem de Erro'])
    writer.writeheader()

    # Carregar arquivo de áudio
    rate, audio_data = ler_arquivo_wav(path, writer)

    print('Áudio original')
    Audio(path)

    print('Convolução do sinal de áudio original')
    audio_conv = convolucao(audio_data, path, writer)
    # Audio(audio_conv)

    print('Normalizar sinal de áudio original')
    audio_norm = normalizar_audio(audio_data, path, writer)
    # Audio(audio_norm)

Áudio original
Convolução do sinal de áudio original
Normalizar sinal de áudio original


In [41]:
## Exibir arquivo de log

with open('..\logs\log_audio.csv', mode='r') as file:

    csv_reader = csv.reader(file)

    for row in csv_reader:
        print(row)

['Data e Hora', 'Tipo de Processamento', 'Tempo de Processamento', 'Nome do Arquivo', 'Tamanho do Arquivo', 'Resultado do Processamento', 'Mensagem de Erro']
['2024-03-10 13:46:02', 'Leitura de Arquivo WAV', '1.0', 'audio_test.wav', '940.19140625', 'Sucesso', '']
['2024-03-10 13:46:02', 'Convolução de Áudio', '', 'audio_test.wav', '940.19140625', 'Erro', 'object too deep for desired array']


## Exemplo de uso
> Criando o arquivo ***log_audio.csv*** para análise 